In [1]:
import csv

class RoundRobin:

    def round_robin(self, input_data, time_quantum):
        process_data = []

        # Read data from CSV file
        for row in input_data:
            process_id = row["Patient"]
            arrival_time = int(row["Arrival Time"])
            burst_time = int(row["Estimated Treatment Time"])
            temporary = [process_id, arrival_time, burst_time, 0, burst_time]
            process_data.append(temporary)

        w_time, t_time = RoundRobin.schedulingProcess(self, process_data, time_quantum)
        return w_time, t_time

    def schedulingProcess(self, process_data, time_quantum):
        start_time = []
        exit_time = []
        executed_process = []
        ready_queue = []
        s_time = 0
        process_data.sort(key=lambda x: x[1])

        while 1:
            normal_queue = []
            temp = []

            for i in range(len(process_data)):
                if process_data[i][1] <= s_time and process_data[i][3] == 0:
                    present = 0

                    if len(ready_queue) != 0:
                        for k in range(len(ready_queue)):
                            if process_data[i][0] == ready_queue[k][0]:
                                present = 1

                    if present == 0:
                        temp.extend([process_data[i][0], process_data[i][1], process_data[i][2], process_data[i][4]])
                        ready_queue.append(temp)
                        temp = []

                    if len(ready_queue) != 0 and len(executed_process) != 0:
                        for k in range(len(ready_queue)):
                            if ready_queue[k][0] == executed_process[len(executed_process) - 1]:
                                ready_queue.insert((len(ready_queue) - 1), ready_queue.pop(k))

                elif process_data[i][3] == 0:
                    temp.extend([process_data[i][0], process_data[i][1], process_data[i][2], process_data[i][4]])
                    normal_queue.append(temp)
                    temp = []

            if len(ready_queue) == 0 and len(normal_queue) == 0:
                break

            if len(ready_queue) != 0:
                if ready_queue[0][2] > time_quantum:
                    start_time.append(s_time)
                    s_time = s_time + time_quantum
                    e_time = s_time
                    exit_time.append(e_time)
                    executed_process.append(ready_queue[0][0])

                    for j in range(len(process_data)):
                        if process_data[j][0] == ready_queue[0][0]:
                            break

                    process_data[j][2] = process_data[j][2] - time_quantum
                    ready_queue.pop(0)
                elif ready_queue[0][2] <= time_quantum:
                    start_time.append(s_time)
                    s_time = s_time + ready_queue[0][2]
                    e_time = s_time
                    exit_time.append(e_time)
                    executed_process.append(ready_queue[0][0])

                    for j in range(len(process_data)):
                        if process_data[j][0] == ready_queue[0][0]:
                            break

                    process_data[j][2] = 0
                    process_data[j][3] = 1
                    process_data[j].append(e_time)
                    ready_queue.pop(0)
            elif len(ready_queue) == 0:
                if s_time < normal_queue[0][1]:
                    s_time = normal_queue[0][1]

                if normal_queue[0][2] > time_quantum:
                    start_time.append(s_time)
                    s_time = s_time + time_quantum
                    e_time = s_time
                    exit_time.append(e_time)
                    executed_process.append(normal_queue[0][0])

                    for j in range(len(process_data)):
                        if process_data[j][0] == normal_queue[0][0]:
                            break

                    process_data[j][2] = process_data[j][2] - time_quantum
                elif normal_queue[0][2] <= time_quantum:
                    start_time.append(s_time)
                    s_time = s_time + normal_queue[0][2]
                    e_time = s_time
                    exit_time.append(e_time)
                    executed_process.append(normal_queue[0][0])

                    for j in range(len(process_data)):
                        if process_data[j][0] == normal_queue[0][0]:
                            break

                    process_data[j][2] = 0
                    process_data[j][3] = 1
                    process_data[j].append(e_time)

        t_time = RoundRobin.calculateTurnaroundTime(self, process_data)
        w_time = RoundRobin.calculateWaitingTime(self, process_data)
        RoundRobin.printData(self, process_data, t_time, w_time, executed_process)
        return w_time, t_time

    def calculateTurnaroundTime(self, process_data):
        total_turnaround_time = 0
        for i in range(len(process_data)):
            turnaround_time = process_data[i][5] - process_data[i][1]
            total_turnaround_time = total_turnaround_time + turnaround_time
            process_data[i].append(turnaround_time)

        average_turnaround_time = total_turnaround_time / len(process_data)
        return average_turnaround_time

    def calculateWaitingTime(self, process_data):
        total_waiting_time = 0
        for i in range(len(process_data)):
            waiting_time = process_data[i][6] - process_data[i][4]
            total_waiting_time = total_waiting_time + waiting_time
            process_data[i].append(waiting_time)

        average_waiting_time = total_waiting_time / len(process_data)
        return average_waiting_time

    def printData(self, process_data, average_turnaround_time, average_waiting_time, executed_process):
        process_data.sort(key=lambda x: x[0])
        print("Process_ID  Arrival_Time  Rem_Burst_Time   Completed  Original_Burst_Time  Completion_Time  Turnaround_Time  Waiting_Time")
        for i in range(len(process_data)):
            for j in range(len(process_data[i])):
                print(process_data[i][j], end="                ")
            print()

        print(f'Average Turnaround Time: {average_turnaround_time}')
        print(f'Average Waiting Time: {average_waiting_time}')
        print(f'Sequence of Processes: {executed_process}')





In [2]:
import csv

class FCFS:
    def fcfs(self, input_data):
        process_data = []

        # Read data from CSV file
        for row in input_data:
            process_id = row["Patient"]
            arrival_time = int(row["Arrival Time"])
            burst_time = int(row["Estimated Treatment Time"])
            process_data.append([process_id, arrival_time, burst_time])

        process_data.sort(key=lambda x: x[1])
        start_time = []
        exit_time = []
        s_time = 0

        for i in range(len(process_data)):
            if s_time < process_data[i][1]:
                s_time = process_data[i][1]
            start_time.append(s_time)
            s_time = s_time + process_data[i][2]
            e_time = s_time
            exit_time.append(e_time)
            process_data[i].append(e_time)

        t_time = self.calculateTurnaroundTime(process_data)
        w_time = self.calculateWaitingTime(process_data)
        self.printData(process_data, t_time, w_time)
        return w_time, t_time

    def calculateTurnaroundTime(self, process_data):
        total_turnaround_time = 0
        for i in range(len(process_data)):
            turnaround_time = process_data[i][3] - process_data[i][1]
            total_turnaround_time = total_turnaround_time + turnaround_time
            process_data[i].append(turnaround_time)
        average_turnaround_time = total_turnaround_time / len(process_data)
        return average_turnaround_time

    def calculateWaitingTime(self, process_data):
        total_waiting_time = 0
        for i in range(len(process_data)):
            waiting_time = process_data[i][4] - process_data[i][2]
            total_waiting_time = total_waiting_time + waiting_time
            process_data[i].append(waiting_time)
        average_waiting_time = total_waiting_time / len(process_data)
        return average_waiting_time

    def printData(self, process_data, average_turnaround_time, average_waiting_time):
        print("Process_ID  Arrival_Time  Burst_Time  Completion_Time  Turnaround_Time  Waiting_Time")
        for i in range(len(process_data)):
            for j in range(len(process_data[i])):
                print(process_data[i][j], end="                ")
            print()
        print(f'Average Turnaround Time: {average_turnaround_time}')
        print(f'Average Waiting Time: {average_waiting_time}\n')



In [3]:
import csv

class SJF:

    def sjf_scheduling(self, input_data):
        process_data = []

        # Read data from CSV file
        for row in input_data:
            process_id = row["Patient"]
            arrival_time = int(row["Arrival Time"])
            burst_time = int(row["Estimated Treatment Time"])
            process_data.append([process_id, arrival_time, burst_time, 0])

        start_time = []
        exit_time = []
        s_time = 0
        process_data.sort(key=lambda x: x[1])

        for i in range(len(process_data)):
            ready_queue = []
            temp = []
            normal_queue = []

            for j in range(len(process_data)):
                if (process_data[j][1] <= s_time) and (process_data[j][3] == 0):
                    temp.extend([process_data[j][0], process_data[j][1], process_data[j][2]])
                    ready_queue.append(temp)
                    temp = []
                elif process_data[j][3] == 0:
                    temp.extend([process_data[j][0], process_data[j][1], process_data[j][2]])
                    normal_queue.append(temp)
                    temp = []

            if len(ready_queue) != 0:
                ready_queue.sort(key=lambda x: x[2])
                start_time.append(s_time)
                s_time = s_time + ready_queue[0][2]
                e_time = s_time
                exit_time.append(e_time)
                for k in range(len(process_data)):
                    if process_data[k][0] == ready_queue[0][0]:
                        break
                process_data[k][3] = 1
                process_data[k].append(e_time)

            elif len(ready_queue) == 0:
                if s_time < normal_queue[0][1]:
                    s_time = normal_queue[0][1]
                start_time.append(s_time)
                s_time = s_time + normal_queue[0][2]
                e_time = s_time
                exit_time.append(e_time)
                for k in range(len(process_data)):
                    if process_data[k][0] == normal_queue[0][0]:
                        break
                process_data[k][3] = 1
                process_data[k].append(e_time)

        t_time = self.calculateTurnaroundTime(process_data)
        w_time = self.calculateWaitingTime(process_data)
        self.printData(process_data, t_time, w_time)
        return w_time, t_time

    def calculateTurnaroundTime(self, process_data):
        total_turnaround_time = 0
        for i in range(len(process_data)):
            turnaround_time = process_data[i][4] - process_data[i][1]
            total_turnaround_time = total_turnaround_time + turnaround_time
            process_data[i].append(turnaround_time)
        average_turnaround_time = total_turnaround_time / len(process_data)
        return average_turnaround_time

    def calculateWaitingTime(self, process_data):
        total_waiting_time = 0
        for i in range(len(process_data)):
            waiting_time = process_data[i][5] - process_data[i][2]
            total_waiting_time = total_waiting_time + waiting_time
            process_data[i].append(waiting_time)
        average_waiting_time = total_waiting_time / len(process_data)
        return average_waiting_time

    def printData(self, process_data, average_turnaround_time, average_waiting_time):
        process_data.sort(key=lambda x: x[0])
        print("Process_ID  Arrival_Time  Burst_Time  Completed  Completion_Time  Turnaround_Time  Waiting_Time")
        for i in range(len(process_data)):
            for j in range(len(process_data[i])):
                print(process_data[i][j], end="\t\t\t")
            print()

        print(f'Average Turnaround Time: {average_turnaround_time}')
        print(f'Average Waiting Time: {average_waiting_time}\n')




In [4]:
class Priority:

    def priority_scheduling(self, input_data):
        process_data = []

        # Convert input data to process_data format
        for row in input_data:
            process_id = row["Patient"]
            arrival_time = int(row["Arrival Time"])
            burst_time = int(row["Estimated Treatment Time"])
            priority = int(row["Urgency Level"])
            process_data.append([process_id, arrival_time, burst_time, priority, 0])

        process_data.sort(key=lambda x: x[1])
        start_time = []
        exit_time = []
        s_time = 0

        for i in range(len(process_data)):
            ready_queue = []
            temp = []
            normal_queue = []

            for j in range(len(process_data)):
                if (process_data[j][1] <= s_time) and (process_data[j][4] == 0):
                    temp.extend([process_data[j][0], process_data[j][1], process_data[j][2], process_data[j][3]])
                    ready_queue.append(temp)
                    temp = []
                elif process_data[j][4] == 0:
                    temp.extend([process_data[j][0], process_data[j][1], process_data[j][2], process_data[j][3]])
                    normal_queue.append(temp)
                    temp = []

            if len(ready_queue) != 0:
                ready_queue.sort(key=lambda x: x[3], reverse=True)
                start_time.append(s_time)
                s_time = s_time + ready_queue[0][2]
                e_time = s_time
                exit_time.append(e_time)
                for k in range(len(process_data)):
                    if process_data[k][0] == ready_queue[0][0]:
                        break
                process_data[k][4] = 1
                process_data[k].append(e_time)
            elif len(ready_queue) == 0:
                if s_time < normal_queue[0][1]:
                    s_time = normal_queue[0][1]
                start_time.append(s_time)
                s_time = s_time + normal_queue[0][2]
                e_time = s_time
                exit_time.append(e_time)
                for k in range(len(process_data)):
                    if process_data[k][0] == normal_queue[0][0]:
                        break
                process_data[k][4] = 1
                process_data[k].append(e_time)

        t_time = self.calculateTurnaroundTime(process_data)
        w_time = self.calculateWaitingTime(process_data)
        self.printData(process_data, t_time, w_time)
        return w_time, t_time

    def calculateTurnaroundTime(self, process_data):
        total_turnaround_time = 0
        for i in range(len(process_data)):
            turnaround_time = process_data[i][5] - process_data[i][1]
            total_turnaround_time = total_turnaround_time + turnaround_time
            process_data[i].append(turnaround_time)
        average_turnaround_time = total_turnaround_time / len(process_data)
        return average_turnaround_time

    def calculateWaitingTime(self, process_data):
        total_waiting_time = 0
        for i in range(len(process_data)):
            waiting_time = process_data[i][6] - process_data[i][2]
            total_waiting_time = total_waiting_time + waiting_time
            process_data[i].append(waiting_time)
        average_waiting_time = total_waiting_time / len(process_data)
        return average_waiting_time

    def printData(self, process_data, average_turnaround_time, average_waiting_time):
        process_data.sort(key=lambda x: x[0])
        print("Process_ID  Arrival_Time  Burst_Time         Priority       Completed  Completion_Time  Turnaround_Time  Waiting_Time")
        for i in range(len(process_data)):
            for j in range(len(process_data[i])):
                print(process_data[i][j], end="\t\t\t\t")
            print()
        print(f'Average Turnaround Time: {average_turnaround_time}')
        print(f'Average Waiting Time: {average_waiting_time}\n')


In [5]:
input_data = [
    {"Patient": "A", "Arrival Time": 0, "Estimated Treatment Time": 30, "Urgency Level": 3},
    {"Patient": "B", "Arrival Time": 10, "Estimated Treatment Time": 20, "Urgency Level": 5},
    {"Patient": "C", "Arrival Time": 15, "Estimated Treatment Time": 40, "Urgency Level": 2},
    {"Patient": "D", "Arrival Time": 20, "Estimated Treatment Time": 15, "Urgency Level": 4},
]
print("FCFS:")
fcfs = FCFS()
fcfs_waiting_time, fcfs_turnaround_time = fcfs.fcfs(input_data)
print("SJF:")
sjf = SJF()
sjf_waiting_time, sjf_turnaround_time = sjf.sjf_scheduling(input_data)
print("PS:")
priority = Priority()
ps_waiting_time, ps_turnaround_time = priority.priority_scheduling(input_data)
print("RR:")
round_robin = RoundRobin()
time_quantum = 4  # Update with the desired time quantum
rr_waiting_time, rr_turnaround_time = round_robin.round_robin(input_data, time_quantum)


FCFS:
Process_ID  Arrival_Time  Burst_Time  Completion_Time  Turnaround_Time  Waiting_Time
A                0                30                30                30                0                
B                10                20                50                40                20                
C                15                40                90                75                35                
D                20                15                105                85                70                
Average Turnaround Time: 57.5
Average Waiting Time: 31.25

SJF:
Process_ID  Arrival_Time  Burst_Time  Completed  Completion_Time  Turnaround_Time  Waiting_Time
A			0			30			1			30			30			0			
B			10			20			1			65			55			35			
C			15			40			1			105			90			50			
D			20			15			1			45			25			10			
Average Turnaround Time: 50.0
Average Waiting Time: 23.75

PS:
Process_ID  Arrival_Time  Burst_Time         Priority       Completed  Completion_Time  Turnaround_Time  Waiting_Time
A	

In [6]:
metrics = {
    "FCFS": (fcfs_waiting_time, fcfs_turnaround_time),
    "SJF": (sjf_waiting_time, sjf_turnaround_time),
    "PS": (ps_waiting_time, ps_turnaround_time),
    "RR": (rr_waiting_time, rr_turnaround_time),
}

best_algorithm = min(metrics, key=lambda k: (metrics[k][0], metrics[k][1]))
print(f"The best scheduling algorithm is: {best_algorithm}")

The best scheduling algorithm is: SJF
